### **Contribution3**

For data munging

\

Fangqing Wu  
001305642



---



# **Contribution Summary**  
This notebook is about data wrangling and data clean and fitting.  
For my contribution, methods for fitting NaN values would be added.  
Way down in "Newly Added Contents" chapter (could be aceessed in left navigation)   






---



## **Newly Added Contents**  

In the before part, values are checked with boxplot for the distribution and outliers detection, also the null-valued feature are filled with mean value.    
In this addtion part/contribution, Random Forests would be used for data filling. 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
  
data = pd.read_csv("https://raw.githubusercontent.com/fwqfwq/dataset/master/cumulative.csv")

A rough view: 

In [ ]:
data.head(5)

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,NaN,NaN,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [ ]:
data = data.iloc[:, :11]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rowid             9564 non-null   int64  
 1   kepid             9564 non-null   int64  
 2   kepoi_name        9564 non-null   object 
 3   kepler_name       2294 non-null   object 
 4   koi_disposition   9564 non-null   object 
 5   koi_pdisposition  9564 non-null   object 
 6   koi_score         8054 non-null   float64
 7   koi_fpflag_nt     9564 non-null   int64  
 8   koi_fpflag_ss     9564 non-null   int64  
 9   koi_fpflag_co     9564 non-null   int64  
 10  koi_fpflag_ec     9564 non-null   int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 822.0+ KB


1. Drop the column 'kepler_name', with the least values. (2294 - generally, 9564)

2. Fit values for 'koi_score'. (8054 - 9564)

1. 

In [ ]:
# Delete 'kepler_name' column
data = data.drop('kepler_name', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rowid             9564 non-null   int64  
 1   kepid             9564 non-null   int64  
 2   kepoi_name        9564 non-null   object 
 3   koi_disposition   9564 non-null   object 
 4   koi_pdisposition  9564 non-null   object 
 5   koi_score         8054 non-null   float64
 6   koi_fpflag_nt     9564 non-null   int64  
 7   koi_fpflag_ss     9564 non-null   int64  
 8   koi_fpflag_co     9564 non-null   int64  
 9   koi_fpflag_ec     9564 non-null   int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 747.3+ KB


2. 

Before training, convert text-valued features to numeric ones. 

In [ ]:
# Check the unique values
print('kepid: ', data['kepoi_name'].unique())
print('kepoi_name: ', data['kepoi_name'].unique())
print('koi_disposition: ', data['koi_disposition'].unique())
print('koi_pdisposition: ', data['koi_pdisposition'].unique())


kepid:  ['K00752.01' 'K00752.02' 'K00753.01' ... 'K07986.01' 'K07987.01'
 'K07989.01']
kepoi_name:  ['K00752.01' 'K00752.02' 'K00753.01' ... 'K07986.01' 'K07987.01'
 'K07989.01']
koi_disposition:  ['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']
koi_pdisposition:  ['CANDIDATE' 'FALSE POSITIVE']


According to the result, drop the 'kepid' and 'kepoi_name', converting others. 

In [ ]:
# Drop cols and change the order
temp_data = data.iloc[:, [5,3,4,6,7,8,9]]
temp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   koi_score         8054 non-null   float64
 1   koi_disposition   9564 non-null   object 
 2   koi_pdisposition  9564 non-null   object 
 3   koi_fpflag_nt     9564 non-null   int64  
 4   koi_fpflag_ss     9564 non-null   int64  
 5   koi_fpflag_co     9564 non-null   int64  
 6   koi_fpflag_ec     9564 non-null   int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 523.2+ KB


In [ ]:
# Convert the values
disposition = {'CONFIRMED': 1, 'FALSE POSITIVE': 2, 'CANDIDATE': 3} 
pdisposition = {'CANDIDATE': 1, 'FALSE POSITIVE': 2} 

temp_data['koi_disposition'] = [disposition[item] for item in temp_data['koi_disposition']] 
temp_data['koi_pdisposition'] = [pdisposition[item] for item in temp_data['koi_pdisposition']] 
temp_data.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,koi_score,koi_disposition,koi_pdisposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec
0,1.000,1,1,0,0,0,0
1,0.969,1,1,0,0,0,0
2,0.000,2,2,0,1,0,0
3,0.000,2,2,0,1,0,0
4,1.000,1,1,0,0,0,0


Extract and split the NaN value for 'koi_scaore'. 

In [ ]:
# notnull() vs isnull()
not_series = pd.notnull(temp_data['koi_score'])
fill_known = temp_data[not_series]

is_series = pd.isnull(temp_data['koi_score'])
fill_unknown = temp_data[is_series]

print('Not NaN:\n', fill_known.head())
print('Is NaN:\n', fill_unknown.head())

Not NaN:
    koi_score  koi_disposition  ...  koi_fpflag_co  koi_fpflag_ec
0      1.000                1  ...              0              0
1      0.969                1  ...              0              0
2      0.000                2  ...              0              0
3      0.000                2  ...              0              0
4      1.000                1  ...              0              0

[5 rows x 7 columns]
Is NaN:
      koi_score  koi_disposition  ...  koi_fpflag_co  koi_fpflag_ec
24         NaN                2  ...              0              0
75         NaN                2  ...              0              1
99         NaN                1  ...              0              0
122        NaN                3  ...              0              0
317        NaN                3  ...              0              0

[5 rows x 7 columns]


Build the RF model for data filling. 

In [ ]:
# Process the train data 
fill_X = fill_known.iloc[:, 1:]
fill_Y = fill_known.iloc[:, 0]
print('X:\n', fill_X.head(), '\n', fill_X.shape)
print('\nY:\n', fill_Y.head(), '\n', fill_Y.shape)

X:
    koi_disposition  koi_pdisposition  ...  koi_fpflag_co  koi_fpflag_ec
0                1                 1  ...              0              0
1                1                 1  ...              0              0
2                2                 2  ...              0              0
3                2                 2  ...              0              0
4                1                 1  ...              0              0

[5 rows x 6 columns] 
 (8054, 6)

Y:
 0    1.000
1    0.969
2    0.000
3    0.000
4    1.000
Name: koi_score, dtype: float64 
 (8054,)


In [ ]:
# Random forests model
fill_rf = RandomForestRegressor(random_state=0, n_estimators=150, max_depth=3, n_jobs=-1)
fill_rf.fit(fill_X, fill_Y)
fill_predict = fill_rf.predict(fill_unknown.iloc[:, 1:]).round(0)

fill_predict

array([0., 0., 1., ..., 1., 0., 0.])

In [ ]:
# Create new filled data
new_result = fill_unknown.iloc[:, 1:] # drop the original Nan col
new_result.insert(0, 'koi_score', fill_predict, True)
new_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1510 entries, 24 to 8932
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   koi_score         1510 non-null   float64
 1   koi_disposition   1510 non-null   int64  
 2   koi_pdisposition  1510 non-null   int64  
 3   koi_fpflag_nt     1510 non-null   int64  
 4   koi_fpflag_ss     1510 non-null   int64  
 5   koi_fpflag_co     1510 non-null   int64  
 6   koi_fpflag_ec     1510 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 94.4 KB


In [ ]:
# Update the dataset
dataset = fill_known.append(new_result)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9564 entries, 0 to 8932
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   koi_score         9564 non-null   float64
 1   koi_disposition   9564 non-null   int64  
 2   koi_pdisposition  9564 non-null   int64  
 3   koi_fpflag_nt     9564 non-null   int64  
 4   koi_fpflag_ss     9564 non-null   int64  
 5   koi_fpflag_co     9564 non-null   int64  
 6   koi_fpflag_ec     9564 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 597.8 KB


All the Nan filled. 



---



## Data cleaning checklist

 * Save original data
 * Identify missing data
 * Identify placeholder data (e.g. 0's for NA's) 
 * Identify outliers
 * Check for overall plausibility and errors (e.g., typos, unreasonable ranges)
 * Identify highly correlated variables
 * Identify variables with (nearly) no variance
 * Identify variables with strange names or values
 * Check variable classes (eg. Characters vs factors)
 * Remove/transform some variables (maybe your model does not like categorial variables)
 * Rename some variables or values (if not all data is useful)
 * Check some overall pattern (statistical/ numerical summaries)
 * Possibly center/scale variables
